In [ ]:
!pip install insightface onnxruntime opencv-python numpy pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1057642 sha256=3b27335c91c7919e9b6aa704f8fdc68cbe0018ab52f20e25a1c5d13c2e3ecdad
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [ ]:
!pip install ttkbootstrap opencv-python insightface pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 33.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Successfully uninstalled pillow-11.2.1


In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import csv
import numpy as np
from datetime import datetime
from insightface.app import FaceAnalysis
from joblib import load

# Load model and label encoder
recognizer = load("/content/face_classifier.joblib")
le = load("/content/label_encoder.joblib")

# Init InsightFace
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

# Paths
test_dir = "/content/drive/MyDrive/testing_dataset"
output_csv = "unlabeled_test_results.csv"
vis_output_dir = "prediction_visuals"
os.makedirs(vis_output_dir, exist_ok=True)

# CSV setup
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Predicted_Label', 'Confidence'])

    # Process each image
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"[WARN] Skipping unreadable image: {img_name}")
            continue

        faces = app.get(img)
        if not faces:
            writer.writerow([img_name, "No face detected", ""])
            continue

        faces.sort(key=lambda x: (x.bbox[2]-x.bbox[0]) * (x.bbox[3]-x.bbox[1]), reverse=True)
        face = faces[0]
        emb = face.embedding.reshape(1, -1)

        probs = recognizer.predict_proba(emb)[0]
        j = np.argmax(probs)
        name = le.classes_[j]
        confidence = probs[j]

        label = f"{name} ({confidence:.2f})" if confidence > 0.75 else "Unknown"
        writer.writerow([img_name, label, f"{confidence:.2f}" if confidence else ""])

        # Draw and save annotated image
        bbox = face.bbox.astype(int)
        cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        cv2.putText(img, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        output_path = os.path.join(vis_output_dir, f"annotated_{img_name}")
        cv2.imwrite(output_path, img)

print(f"✅ Inference complete. Results saved to: {output_csv}")
print(f"🖼️ Annotated images saved in: {vis_output_dir}")


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:02<00:00, 98706.95KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Inference 

In [ ]:
import os
import cv2
import csv
import numpy as np
from datetime import datetime
from insightface.app import FaceAnalysis
from joblib import load

# Load model and label encoder
recognizer = load("/content/face_classifier.joblib")
le = load("/content/label_encoder.joblib")

# Init InsightFace
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

# Paths
test_dir = "/content/drive/MyDrive/testing_dataset"
output_csv = "unlabeled_test_results.csv"
vis_output_dir = "prediction_visuals"
os.makedirs(vis_output_dir, exist_ok=True)

# CSV setup
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Face_ID', 'Predicted_Label', 'Confidence'])

    # Process each image
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"[WARN] Skipping unreadable image: {img_name}")
            continue

        faces = app.get(img)
        if not faces:
            writer.writerow([img_name, "-", "No face detected", ""])
            continue

        for idx, face in enumerate(faces):
            emb = face.embedding.reshape(1, -1)
            probs = recognizer.predict_proba(emb)[0]
            j = np.argmax(probs)
            name = le.classes_[j]
            confidence = probs[j]

            label = f"{name} ({confidence:.2f})" if confidence > 0.75 else "Unknown"
            writer.writerow([img_name, idx + 1, label, f"{confidence:.2f}" if confidence else ""])

            # Draw bounding box and label
            bbox = face.bbox.astype(int)
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
            cv2.putText(img, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Save annotated image
        output_path = os.path.join(vis_output_dir, f"annotated_{img_name}")
        cv2.imwrite(output_path, img)

print(f"✅ Inference complete. Results saved to: {output_csv}")
print(f"🖼️ Annotated images saved in: {vis_output_dir}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Inference 